In [7]:
import os
import torch
import transformers
from transformers import AutoTokenizer
from pydantic import BaseModel, Field
from typing import Type, Optional
import json
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain
from langchain.tools import tool
import requests


In [2]:
MODEL_PATH = 'models/mistral-7b-instruct-v0.1.Q5_0.gguf'  # Replace this with the path to your model

In [3]:
config = {
    "max_new_tokens": 2048,
    "context_length": 4096,
    "repetition_penalty": 1.1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 0.9,
    "stream": True,
    "threads": int(os.cpu_count() / 2)
}

In [4]:
llm = CTransformers(model=MODEL_PATH,
                    config=config)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [8]:
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)

class OutputParser(AgentOutputParser):
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> AgentAction | AgentFinish:
        try:
            # this will work IF the text is a valid JSON with action and action_input
            response = parse_json_markdown(text)
            action, action_input = response["action"], response["action_input"]
            if action == "Final Answer":
                # this means the agent is finished so we call AgentFinish
                return AgentFinish({"output": action_input}, text)
            else:
                # otherwise the agent wants to use an action, so we call AgentAction
                return AgentAction(action, action_input, text)
        except Exception:
            # sometimes the agent will return a string that is not a valid JSON
            # often this happens when the agent is finished
            # so we just return the text as the output
            return AgentFinish({"output": text}, text)

    @property
    def _type(self) -> str:
        return "conversational_chat"


# initialize output parser for agent
parser = OutputParser()

from langchain.agents import initialize_agent

# initialize agent
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=[],
    llm=llm,
    verbose=True,
    early_stopping_method="generate",
    memory=memory,
    agent_kwargs={"output_parser": parser}
)

/Users/suryakiransureshkumar/Documents/Code-bot/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
start_instruct = "<s>[INST] "
end_instruct = "[/INST]"
end_sentence = "</s>"
system_message_plain = """Assistant is a expert in the following functions:
def helloworld():
  return "hello world"

def add(x,y):
  return x+y

Here are some previous conversations between the Assistant and User:
"""
system_message = start_instruct + system_message_plain + end_instruct + end_sentence

In [10]:
system_message

'<s>[INST] Assistant is a expert in the following functions:\ndef helloworld():\n  return "hello world"\n\ndef add(x,y):\n  return x+y\n\nHere are some previous conversations between the Assistant and User:\n[/INST]</s>'

In [11]:
messages = [
        {
            "role": "user",
            "content": 'Hey how are you today?'
        },
        {
            "role": "assistant",
            "content": '''```json
{{"action": "Final Answer",
 "action_input": "I'm good thanks, how can I help you?"}}
```'''
        },
        {
            "role": "user",
            "content": "What does helloworld() do? write the function signature and write an example on how to use it."
        },
            {"role": "assistant",
            "content": '''helloworld takes in no parameters as input and returns a string datatype. you can use it like this  ```python
a = helloworld()
```'''
        }
]

conversation_formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [12]:
prompt = system_message + '\n\n' + conversation_formatted

In [13]:
new_prompt = agent.agent.create_prompt(
    system_message=prompt,
    tools=[]
)

In [14]:
agent.agent.llm_chain.prompt = new_prompt

# instruction = start_instruct + " Respond to the following in JSON with 'action' and 'action_input' values " + end_instruct
human_msg =  "\nUser: {input}"

In [15]:
agent.agent.llm_chain.prompt.messages[2].prompt.template = human_msg

In [16]:
resp = agent("what does add() do?")
resp['output']

/Users/suryakiransureshkumar/Documents/Code-bot/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Assistant: The function `add()` takes two arguments, `x` and `y`, which are integers or floats. It returns the sum of `x` and `y`. Here's an example of how to use it:
```python
result = add(3, 5)
print(result)
```

> Finished chain.


"\nAssistant: The function `add()` takes two arguments, `x` and `y`, which are integers or floats. It returns the sum of `x` and `y`. Here's an example of how to use it:\n```python\nresult = add(3, 5)\nprint(result)\n```"

In [17]:
resp = agent("what did I ask previously?")
resp['output']



> Entering new AgentExecutor chain...

AI: 
Assistant: You asked about the functions `hello world()`, and `add()`. The function `hello world()` returns a string "hello world" with no input parameters. The function `add()` takes two arguments, `x` and `y`, which are integers or floats, and returns the sum of `x` and `y`.

> Finished chain.


'\nAI: \nAssistant: You asked about the functions `hello world()`, and `add()`. The function `hello world()` returns a string "hello world" with no input parameters. The function `add()` takes two arguments, `x` and `y`, which are integers or floats, and returns the sum of `x` and `y`.'